In [1]:
# ----------------------------------------
# 0. 导入必要的库 (与主测试 cell 相同)
# ----------------------------------------
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric
import cv2
# 注意：timm 和自定义的模型类在这里不是必需的，但为了保持环境一致性，我们不移除导入

# ----------------------------------------
# 1. 定义 Baseline "模型"
# ----------------------------------------
class BilinearBaseline(nn.Module):
    """
    一个伪模型，用于测试双线性插值的基准性能。
    它不包含任何可训练的参数。
    它的 forward 方法直接返回输入张量，因为数据集加载器
    已经将低分辨率图像通过双线性插值放大为模型的输入。
    """
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # 直接返回输入，因为输入本身就是双线性插值的结果
        return x

# ----------------------------------------
# 2. 定义 Dataset 类 (与主测试 cell 完全相同)
# ----------------------------------------
class PlainLowHighDataset(Dataset):
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder, self.transform = patches_folder, transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]

    def __len__(self):
        return len(self.fns)

    def __getitem__(self, idx):
        fn = self.fns[idx]
        img_path = os.path.join(self.patches_folder, fn)
        # 加载高分辨率原图
        arr = np.array(Image.open(img_path).convert("L"))
        # 1. 首先下采样，模拟低分辨率输入
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)
        # 2. 然后用双线性插值上采样，这就是我们的 baseline 输出
        up_img = cv2.resize(down_arr, (256, 256), interpolation=cv2.INTER_LINEAR)
        # 将 baseline 的结果和目标（原图）都转换为 Tensor
        inp_t = self.transform(Image.fromarray(up_img)) # 这是 baseline 的预测结果
        tgt_t = self.transform(Image.fromarray(arr))    # 这是真值 (Ground Truth)
        return inp_t, tgt_t, fn # 返回 (预测, 真值, 文件名)

# ----------------------------------------
# 3. 设置路径、参数和 Dataloaders (与主测试 cell 基本相同)
# ----------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.ToTensor()

# ### 新增：为 Baseline 定义一个独立的图像保存文件夹 ###
BASELINE_RECONSTRUCTION_DIR = "lung_bilinear_reconstructions"
IMAGES_TO_SAVE = 5 # 保存的图像数量保持一致
os.makedirs(BASELINE_RECONSTRUCTION_DIR, exist_ok=True)

# 准备 Dataloaders (与之前完全相同)
# 验证集
val_patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_xy"
all_val_fns = sorted([f for f in os.listdir(val_patches_folder) if f.lower().endswith((".jpg", ".png"))])
all_val_indices = list(range(len(all_val_fns)))
_, val_idxs = train_test_split(all_val_indices, test_size=0.25, random_state=42)
val_dataset = PlainLowHighDataset(val_patches_folder, val_idxs, all_val_fns, transform)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# 测试集
test_patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_yz"
all_test_fns = sorted([f for f in os.listdir(test_patches_folder) if f.lower().endswith((".jpg", ".png"))])
all_test_indices = list(range(len(all_test_fns)))
test_dataset = PlainLowHighDataset(test_patches_folder, all_test_indices, all_test_fns, transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# ----------------------------------------
# 4. 实例化 Baseline 模型
# ----------------------------------------
baseline_model = BilinearBaseline().to(device)
baseline_model.eval()
print("成功创建 Bilinear Baseline '模型'。")


# ----------------------------------------
# 5. 定义评估函数 (与主测试 cell 完全相同)
# ----------------------------------------
def tensor_to_image(tensor):
    """将 [0,1] 范围的Tensor转换为可保存的PIL Image对象"""
    arr = tensor.clamp(0, 1).cpu().squeeze().numpy()
    return Image.fromarray((arr * 255).round().astype(np.uint8))

def evaluate_model(loader, model_to_test, output_dir=None, save_count=0):
    total_psnr, total_ssim, count = 0.0, 0.0, 0
    with torch.no_grad():
        for i, (inp_t, tgt_t, fn) in enumerate(loader):
            fn = fn[0] if isinstance(fn, (list, tuple)) else fn
            inp_t, tgt_t = inp_t.to(device), tgt_t.to(device)
            
            # 对于 baseline 模型，out_t 将会与 inp_t 完全相同
            out_t = model_to_test(inp_t)
            
            pred_np = out_t.clamp(0, 1).cpu().squeeze().numpy()
            target_np = tgt_t.cpu().squeeze().numpy()
            
            total_psnr += psnr_metric(target_np, pred_np, data_range=1.0)
            total_ssim += ssim_metric(target_np, pred_np, data_range=1.0, channel_axis=None)
            count += 1
            
            if output_dir is not None and i < save_count:
                base_name = os.path.splitext(fn)[0]
                # 保存三种图像以供对比
                # 在这个 baseline case 中, 'input' 和 'reconstructed' 理论上是完全一样的
                tensor_to_image(inp_t).save(os.path.join(output_dir, f"{base_name}_01_input(bilinear).png"))
                tensor_to_image(out_t).save(os.path.join(output_dir, f"{base_name}_02_reconstructed(bilinear).png"))
                tensor_to_image(tgt_t).save(os.path.join(output_dir, f"{base_name}_03_ground_truth.png"))

    return total_psnr / count, total_ssim / count

# ----------------------------------------
# 6. 执行 Baseline 评估并打印结果
# ----------------------------------------
print(f"\n在 {device} 上评估 Bilinear Interpolation Baseline...")

# 评估验证集
val_psnr, val_ssim = evaluate_model(val_loader, baseline_model)
print(f"\n-> [Baseline] 验证集结果:")
print(f"   - 平均 PSNR: {val_psnr:.4f} dB")
print(f"   - 平均 SSIM: {val_ssim:.4f}")

# 评估测试集，并保存图像
test_psnr, test_ssim = evaluate_model(test_loader, baseline_model, output_dir=BASELINE_RECONSTRUCTION_DIR, save_count=IMAGES_TO_SAVE)
print(f"\n-> [Baseline] 测试集结果:")
print(f"   - 平均 PSNR: {test_psnr:.4f} dB")
print(f"   - 平均 SSIM: {test_ssim:.4f}")
print(f"\n已保存 {IMAGES_TO_SAVE} 组 Baseline 测试图像到 '{BASELINE_RECONSTRUCTION_DIR}/' 文件夹中供查看。")

成功创建 Bilinear Baseline '模型'。

在 cuda 上评估 Bilinear Interpolation Baseline...

-> [Baseline] 验证集结果:
   - 平均 PSNR: 24.0835 dB
   - 平均 SSIM: 0.5669

-> [Baseline] 测试集结果:
   - 平均 PSNR: 23.7104 dB
   - 平均 SSIM: 0.5400

已保存 5 组 Baseline 测试图像到 'lung_bilinear_reconstructions/' 文件夹中供查看。


In [2]:
# ----------------------------------------
# 0. 导入必要的库 (与主测试 cell 相同)
# ----------------------------------------
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric
import cv2

# ----------------------------------------
# 1. 定义一个专门为该 Baseline 服务的 Dataset
# ----------------------------------------
class LowResDataset(Dataset):
    """
    这个 Dataset 与之前的不同，它返回低分辨率图像作为模型输入，
    以及高分辨率图像作为目标。
    """
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder, self.transform = patches_folder, transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]

    def __len__(self):
        return len(self.fns)

    def __getitem__(self, idx):
        fn = self.fns[idx]
        img_path = os.path.join(self.patches_folder, fn)
        # 加载高分辨率原图
        arr = np.array(Image.open(img_path).convert("L"))
        # 1. 下采样以模拟低分辨率输入
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)

        # 将低分辨率图像和高分辨率原图转换为 Tensor
        inp_t = self.transform(Image.fromarray(down_arr)) # 输入是低分辨率图像
        tgt_t = self.transform(Image.fromarray(arr))      # 目标是高分辨率图像
        return inp_t, tgt_t, fn

# ----------------------------------------
# 2. 定义 PixelShuffle Baseline 模型
# ----------------------------------------
class PixelShuffleBaseline(nn.Module):
    """
    一个使用 Conv2d + PixelShuffle 进行升采样的基准模型。
    该模型未经训练，使用随机初始化的权重。
    它将 32x256 的输入放大到 256x256。
    """
    def __init__(self, in_ch=1, out_ch=1, upscale_factor=2):
        super().__init__()
        # 我们需要将高度从 32 放大到 256，即 8 倍。
        # 我们通过连续 3 次 2 倍升采样来实现 (2*2*2=8)
        
        # 第一次升采样: 32x256 -> 64x512
        self.up1 = nn.Sequential(
            nn.Conv2d(in_ch, in_ch * (upscale_factor ** 2), 3, 1, 1),
            nn.PixelShuffle(upscale_factor)
        )
        # 第二次升采样: 64x512 -> 128x1024
        self.up2 = nn.Sequential(
            nn.Conv2d(in_ch, in_ch * (upscale_factor ** 2), 3, 1, 1),
            nn.PixelShuffle(upscale_factor)
        )
        # 第三次升采样: 128x1024 -> 256x2048
        self.up3 = nn.Sequential(
            nn.Conv2d(in_ch, in_ch * (upscale_factor ** 2), 3, 1, 1),
            nn.PixelShuffle(upscale_factor)
        )
        # 最后用一个卷积层将通道数恢复为 out_ch
        self.final_conv = nn.Conv2d(in_ch, out_ch, 3, 1, 1)

    def forward(self, x):
        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.final_conv(x)
        
        # 此刻 x 的尺寸是 [B, C, 256, 2048]
        # 我们需要从宽度上进行中心裁剪，得到 256x256
        _, _, h, w = x.shape
        target_w = 256
        start_w = (w - target_w) // 2
        
        return x[:, :, :, start_w : start_w + target_w]

# ----------------------------------------
# 3. 设置路径、参数和 Dataloaders
# ----------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.ToTensor()

# ### 新增：为 PixelShuffle Baseline 定义一个独立的图像保存文件夹 ###
PIXELSHUFFLE_RECONSTRUCTION_DIR = "lung_pixelshuffle_reconstructions"
IMAGES_TO_SAVE = 5
os.makedirs(PIXELSHUFFLE_RECONSTRUCTION_DIR, exist_ok=True)

# 准备 Dataloaders (使用新的 LowResDataset)
# 验证集
val_patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_xy"
all_val_fns = sorted([f for f in os.listdir(val_patches_folder) if f.lower().endswith((".jpg", ".png"))])
all_val_indices = list(range(len(all_val_fns)))
_, val_idxs = train_test_split(all_val_indices, test_size=0.25, random_state=42)
val_dataset = LowResDataset(val_patches_folder, val_idxs, all_val_fns, transform) # <-- 使用新 Dataset
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# 测试集
test_patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_yz"
all_test_fns = sorted([f for f in os.listdir(test_patches_folder) if f.lower().endswith((".jpg", ".png"))])
all_test_indices = list(range(len(all_test_fns)))
test_dataset = LowResDataset(test_patches_folder, all_test_indices, all_test_fns, transform) # <-- 使用新 Dataset
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# ----------------------------------------
# 4. 实例化 Baseline 模型
# ----------------------------------------
pixelshuffle_baseline_model = PixelShuffleBaseline(in_ch=1, out_ch=1).to(device)
pixelshuffle_baseline_model.eval()
print("成功创建 PixelShuffle Baseline 模型 (权重为随机初始化)。")


# ----------------------------------------
# 5. 定义评估函数 (与主测试 cell 完全相同, 无需修改)
# ----------------------------------------
def tensor_to_image(tensor):
    arr = tensor.clamp(0, 1).cpu().squeeze().numpy()
    return Image.fromarray((arr * 255).round().astype(np.uint8))

def evaluate_model(loader, model_to_test, output_dir=None, save_count=0):
    total_psnr, total_ssim, count = 0.0, 0.0, 0
    with torch.no_grad():
        # 注意：这里 inp_t 是低分辨率图像
        for i, (inp_t, tgt_t, fn) in enumerate(loader):
            fn = fn[0] if isinstance(fn, (list, tuple)) else fn
            inp_t, tgt_t = inp_t.to(device), tgt_t.to(device)
            
            # 模型进行升采样
            out_t = model_to_test(inp_t)
            
            pred_np = out_t.clamp(0, 1).cpu().squeeze().numpy()
            target_np = tgt_t.cpu().squeeze().numpy()
            
            total_psnr += psnr_metric(target_np, pred_np, data_range=1.0)
            total_ssim += ssim_metric(target_np, pred_np, data_range=1.0, channel_axis=None)
            count += 1
            
            if output_dir is not None and i < save_count:
                base_name = os.path.splitext(fn)[0]
                # 为了对比，我们将低分辨率输入也用双线性插值放大后保存
                inp_np_lowres = inp_t.cpu().squeeze().numpy()
                inp_np_resized = cv2.resize(inp_np_lowres, (256, 256), interpolation=cv2.INTER_LINEAR)
                Image.fromarray((inp_np_resized * 255).round().astype(np.uint8)).save(os.path.join(output_dir, f"{base_name}_01_input_lowres_resized.png"))
                tensor_to_image(out_t).save(os.path.join(output_dir, f"{base_name}_02_reconstructed(pixelshuffle).png"))
                tensor_to_image(tgt_t).save(os.path.join(output_dir, f"{base_name}_03_ground_truth.png"))

    return total_psnr / count, total_ssim / count

# ----------------------------------------
# 6. 执行 Baseline 评估并打印结果
# ----------------------------------------
print(f"\n在 {device} 上评估 PixelShuffle Baseline...")

# 评估验证集
val_psnr, val_ssim = evaluate_model(val_loader, pixelshuffle_baseline_model)
print(f"\n-> [Baseline] PixelShuffle 验证集结果:")
print(f"   - 平均 PSNR: {val_psnr:.4f} dB")
print(f"   - 平均 SSIM: {val_ssim:.4f}")

# 评估测试集，并保存图像
test_psnr, test_ssim = evaluate_model(
    test_loader, 
    pixelshuffle_baseline_model, 
    output_dir=PIXELSHUFFLE_RECONSTRUCTION_DIR, 
    save_count=IMAGES_TO_SAVE
)
print(f"\n-> [Baseline] PixelShuffle 测试集结果:")
print(f"   - 平均 PSNR: {test_psnr:.4f} dB")
print(f"   - 平均 SSIM: {test_ssim:.4f}")
print(f"\n已保存 {IMAGES_TO_SAVE} 组 PixelShuffle Baseline 测试图像到 '{PIXELSHUFFLE_RECONSTRUCTION_DIR}/' 文件夹中供查看。")

成功创建 PixelShuffle Baseline 模型 (权重为随机初始化)。

在 cuda 上评估 PixelShuffle Baseline...

-> [Baseline] PixelShuffle 验证集结果:
   - 平均 PSNR: 12.3612 dB
   - 平均 SSIM: 0.0385

-> [Baseline] PixelShuffle 测试集结果:
   - 平均 PSNR: 12.0378 dB
   - 平均 SSIM: 0.0348

已保存 5 组 PixelShuffle Baseline 测试图像到 'lung_pixelshuffle_reconstructions/' 文件夹中供查看。
